In [64]:
#@title Create arms
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import random
k = 10    #number of arms
T = 1000  #horizon  

# creating limits for each arm
bandits = set()
while len(bandits) < 10:
    a, b = np.sort(np.random.rand(2))
    bandits.add((a, b))
bandits = [[x,y] for x, y in bandits]

# expected value of each arm
expected_val = [np.mean([x,y]) for x, y in bandits]

# best arm
best = np.amax(expected_val)

In [ ]:
#@title Run UCB




u_mi = np.zeros((k,))                   # arm μι estimate
u_bandit_score = np.zeros((k,))         #total score of each arm for first N rounds
u_pulls = np.zeros((k,))                #num of arm pulls
u_inst_score = np.zeros((T,))           #reward for round t
u_best_score = np.zeros((T,))           #cumulative reward of best arm for round t
u_alg_score = np.zeros((T,))            #cumulative reward for round t
u_regret =  np.zeros((T,))              #regret for round t
u_times_played = np.zeros((k,))         # num of times armed is played
ucb= np.full(k, float("inf"))           # ucb estimate= μi + square_root(lnT/Qi(T))



# Exploit arm with the best ucb

for i in range(T):

  #finding arm with the best ucb index
  j= np.argmax(ucb)

  #exploiting the arm and store its score
  score= np.random.binomial(1,p=bandit[j])
  u_inst_score[i] = score
  u_bandit_score[j] += score
  u_times_played[j] += 1

     # maintain estimate μι and ucb of each arm
  u_mi[j]= u_bandit_score[j]/u_times_played[j]
  ucb[j]= u_mi[j] + np.sqrt(np.log(T)/u_times_played[j])
  #print('round= %f arm played %f, ucb=%f, Q affecting ucb=%f'%( i,j,ucb[j], (ucb[j]-mi[j])/ucb[j]))



In [ ]:
#@title Run e-Greedy

e_mi = np.zeros((k,))                     # arm estimate
e_bandit_score = np.zeros((k,))           #total score of each arm for first N rounds
e_pulls = np.zeros((k,))                  #num of arm pulls
e_inst_score = np.zeros((T,))             #reward for round t
e_best_score = np.zeros((T,))             #cumulative reward of best arm for round t
e_alg_score = np.zeros((T,))              #cumulative reward for round t
e_regret =  np.zeros((T,))                #regret for round t
e_times_played = np.zeros((k,))           #num of times armed is played
counter= 0


# e-Greedy code
for i in range (T):
  
   # Calculating the exploring probability epsilon= t^(-1/3)
   if i==0:
     eps=0
   else:
     eps=np.power(i, -1/3) * np.power((k*np.log2(i)), 1/3)
   p= random.random()
   
   #Choose when to explore or exploit based on an experiment 
   # Explore with prob epsilon
   if p < eps:
        j = random.randint(0, len(bandit) - 1)   
        
   # Exploit best arm  with probability 1-epsilon   
   else:
        j = np.argmax(e_mi)

   # play the arm and store each new score
   score= np.random.binomial(1,p=bandit[j])
   e_inst_score[i] = score
   e_bandit_score[j] += score
   e_times_played[j] += 1

   # maintain estimate of each arm
   e_mi[j]= e_bandit_score[j]/e_times_played[j]


In [ ]:
#@title Print e-Greedy Cumulatige Regret-T

for i in range(k):
   print('arm = %d: true mean = %f : estimate mean = %f' % (i,bandit[i],e_mi[i]))

for i in range(T):
  if i > 0: e_best_score[i] = e_best_score[i-1] + best #vector keeping track of t*optimal reward (cummulative reward)
  else: e_best_score[i] = best
  if i > 0: e_alg_score[i] = e_alg_score[i-1] + e_inst_score[i] #vector keeping track of cummulative e-Greedy reward at all times 
  else: e_alg_score[i] = e_inst_score[i]
  e_regret[i] = (e_best_score[i] - e_alg_score[i])/(i+1)  #regret per iteration at round t


plt.title("e-Greedy Performance") 
plt.xlabel("Round T") 
plt.ylabel("Total score") 
plt.plot(np.arange(1,T+1),e_regret) 
plt.show()

In [ ]:
#@title Print UCB Cumulative Regret-T

for i in range(k):
   print('arm = %d: true mean = %f : estimate mean = %f' % (i,bandit[i],u_mi[i]))


for i in range(T):
  if i > 0: u_best_score[i] = u_best_score[i-1] + best #vector keeping track of t*optimal reward (cummulative reward)
  else: u_best_score[i] = best
  if i > 0: u_alg_score[i] = u_alg_score[i-1] + u_inst_score[i] #vector keeping track of UCB reward at all times 
  else: u_alg_score[i] = u_inst_score[i]
  u_regret[i] = (u_best_score[i] - u_alg_score[i])/(i+1)  #regret per iteration at round t

plt.title("UCB Performance") 
plt.xlabel("Round T") 
plt.ylabel("Total score") 
plt.plot(np.arange(1,T+1),u_regret) 
plt.show()

In [ ]:
#@title Plot both UCB and e-Greedy regrets




for i in range(T):
  if i > 0: u_best_score[i] = u_best_score[i-1] + best #vector keeping track of t*optimal reward (cummulative reward)
  else: u_best_score[i] = best
  if i > 0: u_alg_score[i] = u_alg_score[i-1] + u_inst_score[i] #vector keeping track of UCB reward at all times 
  else: u_alg_score[i] = u_inst_score[i]
  u_regret[i] = (u_best_score[i] - u_alg_score[i])/(i+1)  #regret per iteration at round t


for i in range(T):
  if i > 0: e_best_score[i] = e_best_score[i-1] + best #vector keeping track of t*optimal reward (cummulative reward)
  else: e_best_score[i] = best
  if i > 0: e_alg_score[i] = e_alg_score[i-1] + e_inst_score[i] #vector keeping track of cummulative e-Greedy reward at all times 
  else: e_alg_score[i] = e_inst_score[i]
  e_regret[i] = (e_best_score[i] - e_alg_score[i])/(i+1)  #regret per iteration at round t


plt.title("ε-Greedy vs UCB Performance")
plt.xlabel("Round T")
plt.ylabel("Total score")
plt.plot(np.arange(1,T+1),u_regret, label="UCB") 
plt.plot(np.arange(1,T+1),e_regret, label="ε-Greedy")
plt.legend()
plt.show()

